In [6]:
import requests
import pandas as pd

url = "https://api.gdc.cancer.gov/cases"

params = {
    "filters": {
        "op": "in",
        "content": {
            "field": "project.project_id",
            "value": ["TCGA-LUAD"]
        }
    },
    "fields": "case_id,submitter_id",
    "format": "JSON",
    "size": "20000"     # large enough to retrieve all cases
}

response = requests.post(url, json=params)

if response.status_code != 200:
    raise Exception("API request failed:", response.text)

data = response.json()

# Extract cases
records = data["data"]["hits"]
df = pd.json_normalize(records)

# Rename and keep clean table
df = df.rename(columns={"case_id": "Case UUID", "submitter_id": "Subject ID"})
df = df[["Subject ID", "Case UUID"]].drop_duplicates()
df.to_excel("/lab/deasylab3/Himangi/psg/TCGA-LUAD-subjID.xlsx", index=False)
print("✅ Saved: TCGA-LUAD-subjID.xlsx")


ModuleNotFoundError: No module named 'openpyxl'

In [7]:
df

,Subject ID,Case UUID
0,TCGA-MP-A4T9,70081320-540f-41d2-8687-ee6d011f8eb0
1,TCGA-MP-A4SV,706420c4-8820-4b41-80d9-f3efd1d2a4f0
2,TCGA-55-8621,8d535aa6-6d59-4965-b5ed-17217d8f5930
3,TCGA-17-Z003,c390b6ad-14c8-49ea-9b38-798c9fad5b62
4,TCGA-50-5051,e1c55bac-bb41-4964-add5-ecd6ba04b58d
...,...,...
580,TCGA-55-6543,e68219b0-a9c2-49df-8f0d-db5ea97fd2dc
581,TCGA-44-8117,6d96a298-be5a-4e14-afe8-ca4b6ca298d6
582,TCGA-05-4432,e737f650-b72d-44e7-b750-558a56716803
583,TCGA-44-3918,6e3b6b72-142d-4b8d-a462-28a205796e41


In [12]:
import os
import pandas as pd
import requests

        # <-- UPDATED
SUBJECT_COLUMN = "Subject ID"
OUTPUT_DIR = "/lab/deasylab3/Jung/Data/Shared_Data/TCGA_Lung/pathology-slides"
os.makedirs(OUTPUT_DIR, exist_ok=True)

df = df
subject_ids = df[SUBJECT_COLUMN].dropna().unique().tolist()

def get_slide_file_ids(case_id):
    url = "https://api.gdc.cancer.gov/files"
    params = {
        "filters": {
            "op": "and",
            "content": [
                {"op": "in", "content": {"field": "cases.submitter_id", "value": [case_id]}},
                {"op": "in", "content": {"field": "data_format", "value": ["SVS"]}},
                {"op": "in", "content": {"field": "experimental_strategy", "value": ["Diagnostic Slide"]}}
            ]
        },
        "fields": "file_id,file_name",
        "format": "JSON",
        "size": "2000"
    }

    r = requests.post(url, json=params)
    if r.status_code == 200:
        data = r.json()
        return [(f["file_id"], f["file_name"]) for f in data["data"]["hits"]]
    return []

def download_slide(file_id, filename):
    url = f"https://api.gdc.cancer.gov/data/{file_id}"
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(filename, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"✅ Downloaded: {filename}")
    else:
        print(f"❌ Failed to download {file_id}")

for case_id in subject_ids:
    print(f"🔍 Searching slides for: {case_id}")
    slides = get_slide_file_ids(case_id)
    if not slides:
        print(f"⚠️ No slides found for {case_id}")
        continue
    for file_id, file_name in slides:
        dst = os.path.join(OUTPUT_DIR, file_name)
        if not os.path.exists(dst):
            download_slide(file_id, dst)
        else:
            print(f"⏩ Already exists: {file_name}")


🔍 Searching slides for: TCGA-MP-A4T9
⏩ Already exists: TCGA-MP-A4T9-01Z-00-DX1.F7B341C4-EBCD-455F-BE90-3B77AC6B76EC.svs
🔍 Searching slides for: TCGA-MP-A4SV
⏩ Already exists: TCGA-MP-A4SV-01Z-00-DX1.5430F2CF-BE4B-42D6-9CD4-92E6FB3F6E99.svs
🔍 Searching slides for: TCGA-55-8621
⏩ Already exists: TCGA-55-8621-01Z-00-DX1.7C519007-D59D-432A-BF4D-23D14A1C8BB6.svs
🔍 Searching slides for: TCGA-17-Z003
⚠️ No slides found for TCGA-17-Z003
🔍 Searching slides for: TCGA-50-5051
⚠️ No slides found for TCGA-50-5051
🔍 Searching slides for: TCGA-62-8394
⚠️ No slides found for TCGA-62-8394
🔍 Searching slides for: TCGA-67-3773
⏩ Already exists: TCGA-67-3773-01Z-00-DX1.3E9DFC22-E962-4C18-BF5B-27EBEA089F5D.svs
🔍 Searching slides for: TCGA-17-Z038
⚠️ No slides found for TCGA-17-Z038
🔍 Searching slides for: TCGA-MN-A4N1
⏩ Already exists: TCGA-MN-A4N1-01Z-00-DX2.9B0852C4-16BF-4962-B86F-E2570E48A89E.svs
⏩ Already exists: TCGA-MN-A4N1-01Z-00-DX1.07CD10C3-73A7-47E5-AD45-45458CE33107.svs
🔍 Searching slides for: T

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))